# Web Scraping Notebook
## NBA Player Career Stats

In [2]:
from bs4 import BeautifulSoup
from bs4 import SoupStrainer
import pandas as pd
import time
import requests

In [3]:
f = open("players.txt", "r")
ply_list = f.read().split("\n")
print(ply_list)

['Abdelnaby, Alaa', 'Abdul-Aziz, Zaid', 'Abdul-Jabbar, Kareem', 'Abdul-Rauf, Mahmoud', 'Abdul-Wahad, Tariq', 'Abdur-Rahim, Shareef', 'Abernethy, Tom', 'Able, Forest', 'Abramovic, John', 'Abrines, Alex', 'Acker, Alex', 'Ackerman, Donald', 'Acres, Mark', 'Acton, Charles', 'Acy, Quincy', 'Adams, Alvan', 'Adams, Don', 'Adams, Hassan', 'Adams, Jaylen', 'Adams, Jordan', 'Adams, Michael', 'Adams, Steven', 'Addison, Rafael', 'Adebayo, Bam', 'Adel, Deng', 'Adelman, Rick', 'Adrien, Jeff', 'Afflalo, Arron', 'Ager, Maurice', 'Aguirre, Mark', 'Ahearn, Blake', 'Ainge, Danny', 'Ajinca, Alexis', 'Akin, Henry', 'Akognon, Josh', 'Akoon-Purcell, DeVaughn', 'Alabi, Solomon', 'Alarie, Mark', 'Alcorn, Gary', 'Aldemir, Furkan', 'Aldrich, Cole', 'Aldridge, LaMarcus', 'Aleksinas, Chuck', 'Alexander-Walker, Nickeil', 'Alexander, Cliff', 'Alexander, Cory', 'Alexander, Courtney', 'Alexander, Gary', 'Alexander, Joe', 'Alexander, Kyle', 'Alexander, Victor', 'Alford, Steve', 'Alkins, Rawle', 'Allen, Bob', 'Allen, Gr

In [4]:
root = "https://www.basketball-reference.com/players/"
players = pd.DataFrame({"fname":[],
                        "lname":[],
                        "g":[],
                        "pts":[],
                        "trb":[],
                        "ast":[],
                        "fgp":[],
                        "fg3p":[],
                        "ftp":[],
                        "efgp":[],
                        "per":[],
                        "ws":[],
                         })
players.head()

,fname,lname,g,pts,trb,ast,fgp,fg3p,ftp,efgp,per,ws


In [6]:
# Ending pattern = 'last initial' + / + 'last[:5]' + 'first[:2]' + '01.html'
# Eg. j/jordami01.html

def rooter(item):
    sub = item.split(",")
    add = str(sub[0][0]) + "/" + str(sub[0][:5]) +  str(sub[1][:3]) + "01.html"
    add = add.replace(" ","").lower()
    return(str(root + add))
print(rooter('Stojakovic, Peja'))

https://www.basketball-reference.com/players/s/stojape01.html


In [7]:
def plyr_scrape(name, y):
    url = rooter(name)
    name = name.split(", ")
    fname = name[1]
    lname = name[0]
    headers = requests.utils.default_headers()
    req = requests.get(url, headers)
    soup = BeautifulSoup(req.content, 'html.parser')
    page = soup.prettify()
    page1 = soup.findAll("div", {"class": "p1"})
    page2 = soup.findAll("div", {"class": "p2"})
    page3 = soup.findAll("div", {"class": "p3"})
    x = 1
    page1 = str(page1).split("</div>")
    page2 = str(page2).split("</div>")
    page3 = str(page3).split("</div>")
    for n in page1:
        if "Games" in n:
            n = str(n).split("</h4><p></p>")
            g = n[1]
            g = str(g).replace("<p>","")
            g = str(g).replace("</p>","")
            g = str(g).replace("\n","")
        elif "Points" in n:
            n = str(n).split("</h4><p></p>")
            pts = n[1]
            pts = str(pts).replace("<p>","")
            pts = str(pts).replace("</p>","")
            pts = str(pts).replace("\n","")
        elif "Total Rebounds" in n:
            n = str(n).split("</h4><p></p>")
            trb = n[1]
            trb = str(trb).replace("<p>","")
            trb = str(trb).replace("</p>","")
            trb = str(trb).replace("\n","")
        elif "Assists" in n:
            n = str(n).split("</h4><p></p>")
            ast = n[1]
            ast = str(ast).replace("<p>","")
            ast = str(ast).replace("</p>","")
            ast = str(ast).replace("\n","")
    x = 1
    for n in str(page2).split("</h4><p></p>"):
        if x == 1:
            x += 1
            continue
        elif x == 2:            
            p = str(n).split("</p>',")
            fgp = p[0]
            fgp = str(fgp)[5:]
            x += 1
            continue
        elif x == 3:            
            p = str(n).split("</p>',")
            fg3p = p[0]
            fg3p = str(fg3p)[5:]
            x += 1
            continue
        elif x == 4:
            x += 1
            p = str(n).split("</p>',")
            ftp = p[0]
            ftp = str(ftp)[5:]
            continue
        elif x == 5:
            x += 1
            p = str(n).split("</p>',")
            efgp = p[0]
            efgp = str(efgp)[5:]
            continue
    x = 1
    for n in str(page3).split("</h4><p></p>"):
        if x == 1:
            x += 1
            continue
        elif x == 2:
            x += 1
            p = str(n).split("</p>',")
            per = p[0]
            per = str(per)[5:]
        elif x == 3:
            ws = n.split("</p>")
            ws = str(ws[0])[5:]
            x += 1
            continue
    df = pd.DataFrame({"fname":[fname],
                        "lname":[lname],
                        "g":[g],
                        "pts":[pts],
                        "trb":[trb],
                        "ast":[ast],
                        "fgp":[fgp],
                        "fg3p":[fg3p],
                        "ftp":[ftp],
                        "efgp":[efgp],
                        "per":[per],
                        "ws":[ws],
                         },index = [y])
    return df
    
print(plyr_scrape(ply_list[0], 1))

  fname      lname    g  pts  trb  ast   fgp fg3p   ftp  efgp   per   ws
1  Alaa  Abdelnaby  256  5.7  3.3  0.3  50.2  0.0  70.1  50.2  13.0  4.8


In [8]:
y = 1
for player in ply_list:
    try:
        insert = plyr_scrape(player, y)
    except:
        continue
    players = players.append(insert)
    y += 1
players.head()

,fname,lname,g,pts,trb,ast,fgp,fg3p,ftp,efgp,per,ws
1,Alaa,Abdelnaby,256,5.7,3.3,0.3,50.2,0.0,70.1,50.2,13.0,4.8
2,Kareem,Abdul-Jabbar,1560,24.6,11.2,3.6,55.9,5.6,72.1,55.9,24.6,273.4
3,Tariq,Abdul-Wahad,236,7.8,3.3,1.1,41.7,23.7,70.3,42.2,11.4,3.5
4,Shareef,Abdur-Rahim,830,18.1,7.5,2.5,47.2,29.7,81.0,47.9,19.0,71.2
5,Tom,Abernethy,319,5.6,3.2,1.2,49.2,0.0,74.7,49.2,12.9,13.4


In [9]:
players.head(400)

,fname,lname,g,pts,trb,ast,fgp,fg3p,ftp,efgp,per,ws
1,Alaa,Abdelnaby,256,5.7,3.3,0.3,50.2,0.0,70.1,50.2,13.0,4.8
2,Kareem,Abdul-Jabbar,1560,24.6,11.2,3.6,55.9,5.6,72.1,55.9,24.6,273.4
3,Tariq,Abdul-Wahad,236,7.8,3.3,1.1,41.7,23.7,70.3,42.2,11.4,3.5
4,Shareef,Abdur-Rahim,830,18.1,7.5,2.5,47.2,29.7,81.0,47.9,19.0,71.2
5,Tom,Abernethy,319,5.6,3.2,1.2,49.2,0.0,74.7,49.2,12.9,13.4
6,Alex,Abrines,174,5.3,1.4,0.5,38.7,36.8,88.0,52.5,8.8,5.0
7,Alex,Acker,30,2.7,1.0,0.5,37.0,32.0,50.0,41.3,8.5,-0.2
8,Mark,Acres,375,3.6,4.1,0.5,50.6,53.8,66.5,50.9,9.0,9.3
9,Quincy,Acy,337,4.9,3.5,0.6,44.4,35.0,75.9,51.3,11.2,8.7
10,Alvan,Adams,988,14.1,7.0,4.1,49.8,13.3,78.8,49.8,18.3,73.5


In [10]:
players.to_csv("scrape.csv")